### Uses Paper Trade - test money
##### https://alpaca.markets/docs/api-documentation/api-v2/

In [1]:
import requests, json
import time, datetime

from paper_config import *
import alpha, database

DataStore = database.DataStore()

C:\Users\albeadam\Miniconda3\envs\ml4t\lib\site-packages\arctic\_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
C:\Users\albeadam\Miniconda3\envs\ml4t\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
def get_account():
    r = requests.get(ALPACA_ACCOUNT_URL, headers=HEADERS)
    return json.loads(r.content)

def create_order(symbol, qty, side, type, time_in_force):
    data = {
        'symbol': symbol,
        'qty': qty,
        'side': side,
        'type': type,
        'time_in_force': time_in_force
    }
    r = request.post(ORDERS_URL, json=data, header=HEADERS)
    return json.loads(r.content)
    
def get_orders():
    r = requests.get(ORDERS_URL, header=HEADERS)
    return json.loads(r.content)

In [3]:
def populate_popular_stocks_high_usage_alphas(interval):
    # test of AAPL, TSLA, AMZN, AMD, FB, NFLX using High Usage
    stocks = ['AAPL', 'TSLA', 'AMZN', 'AMD', 'FB', 'NFLX']
    toc = alpha.get_toc()
    
    high_indicators = []
    for indicator in toc:
        if 'High Usage' in indicator:
            name = indicator.replace(' High Usage', '')
            high_indicators.append(name)
    
    status = 1
    for stock in stocks:
        for indicator in high_indicators:
            status = DataStore.add(stock, indicator, interval)
            if status == -1:
                break
        if status == -1:
            break
    if status == -1:
        time.sleep(60)
        populate_popular_stocks_high_usage_alphas(interval)
        


In [6]:
x,y = DataStore.read('AAPL_OBV_daily')

In [15]:
key = [k for k in y.keys() if 'Last Refreshed' in k]
last_refreshed = y[key[0]]
dates = list(x.keys())
dates.sort(key = lambda date: datetime.datetime.strptime(date, '%Y-%m-%d'), reverse=True)
need_refreshing = dates[:dates.index(last_refreshed)+1]
need_refreshing

['2021-06-14']

In [10]:
key = [k for k in y.keys() if 'Last Refreshed' in k]
last_refreshed = y[key[0]]
dates = list(x.keys())
dates.sort(key = lambda date: datetime.datetime.strptime(date, '%Y-%m-%d'), reverse=True)
need_refreshing = dates[:dates.index(last_refreshed)+1]
need_refreshing

# everything after this data, including this data, will need to be replaced

# first remove this date (using Arctic)

# then append from the API call this date and dates after, will have to use similar method to get dates on or after...
datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')

'2021-06-15'

In [12]:
def update_symbol(stock, indicator, interval):
    DataStore.delete(stock + '_' + indicator + '_' + interval)
    status = DataStore.add(stock, indicator, interval)
    if status == -1:
        time.sleep(60)
        update_symbol(stock, indicator, interval)

def update(sym='all', indicator='all', interval='daily'): # move to database.py, add self, change DataStore to self.
    stores = DataStore.get_symbols()
    updates = 0
    for store in stores:
        intr = store[store.find('_', store.find('_')+1)+1:]
        ind = store[store.find('_')+1:store.find('_', store.find('_')+1)]
        s = store[:store.find('_')]
        if sym == 'all' and indicator == 'all':
            if intr == interval:
                updates += 1
        elif sym == 'all':
            if ind == indicator and intr == interval:
                updates += 1
        elif indicator == 'all':
            if s == sym and intr == interval:
                updates += 1
        elif s == sym and intr == interval and ind == indicator:
            updates += 1
            
    print(f'Approximate time to update {updates} symbols = {(updates/5)-1} minutes')
        
    for store in stores:
        intr = store[store.find('_', store.find('_')+1)+1:]
        ind = store[store.find('_')+1:store.find('_', store.find('_')+1)]
        s = store[:store.find('_')]
        if sym == 'all' and indicator == 'all':
            if intr == interval:
                update_symbol(s, ind, intr)
        elif sym == 'all':
            if ind == indicator and intr == interval:
                update_symbol(s, ind, intr)
        elif indicator == 'all':
            if s == sym and intr == interval:
                update_symbol(s, ind, intr)
        elif s == sym and intr == interval and ind == indicator:
            update_symbol(s, ind, intr)

AAPL ADX daily
AAPL AD daily
AAPL AROON daily
AAPL BBANDS daily
AAPL CCI daily
AAPL EMA daily
AAPL MACD daily
AAPL OBV daily
AAPL RSI daily
AAPL SMA daily
AAPL STOCH daily
AMD ADX daily
AMD AD daily
AMD AROON daily
AMD BBANDS daily
AMD CCI daily
AMD EMA daily
AMD MACD daily
AMD OBV daily
AMD RSI daily
AMD SMA daily
AMD STOCH daily
AMZN ADX daily
AMZN AD daily
AMZN AROON daily
AMZN BBANDS daily
AMZN CCI daily
AMZN EMA daily
AMZN MACD daily
AMZN OBV daily
AMZN RSI daily
AMZN SMA daily
AMZN STOCH daily
FB ADX daily
FB AD daily
FB AROON daily
FB BBANDS daily
FB CCI daily
FB EMA daily
FB MACD daily
FB OBV daily
FB RSI daily
FB SMA daily
FB STOCH daily
NFLX ADX daily
NFLX AD daily
NFLX AROON daily
NFLX BBANDS daily
NFLX CCI daily
NFLX EMA daily
NFLX MACD daily
NFLX OBV daily
NFLX RSI daily
NFLX SMA daily
NFLX STOCH daily
TSLA ADX daily
TSLA AD daily
TSLA AROON daily
TSLA BBANDS daily
TSLA CCI daily
TSLA EMA daily
TSLA MACD daily
TSLA OBV daily
TSLA RSI daily
TSLA SMA daily
TSLA STOCH daily


In [22]:
x.pop('dfd', None)

In [14]:
x = x.pop('5.1: Fast Period', None)

In [15]:
x

12